In [78]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import random
import math

In [2]:
WN = np.array([[0.25,0.5,0.25],
               [0.5,0.1,0.4],
               [0.25,0.4,0.35]])
pop = np.array([100,60,120])

In [4]:
### One ensemble

In [100]:
num_fips = len(pop)
T = 10

In [58]:
x_cutoff = 100
r = 50 ## parameters (in the paper of Lloyd smith is the k) 0.1 to start 
R0 = 2.5  ## the model input
p = r/(R0+r)

weights = np.zeros(x_cutoff)
for i in range(x_cutoff):
    temp1=SS.gamma(r+i)/SS.gamma(r)/SS.gamma((i+1))*np.power(p,r)*np.power((1-p),i)
    weights[i] = temp1

In [59]:
print('it covers', np.sum(weights)*100,'% of the values') ## to generate the sample, the sum up probe should be 1
## we can assign/compare the rest of prob to each one
prob_res = (1- np.sum(weights))/x_cutoff
print (prob_res, 'compare with the last prob', weights[-1])
## normalise the weights to 1
weights_n = weights/np.sum(weights)

it covers 99.99999999999999 % of the values
1.1102230246251566e-18 compare with the last prob 4.946889715640957e-93


In [ ]:
Z = 3 # latent period
Zb = 1 # scale parameter for Z
D = 5 # infectious period
Db = 1 # scale parameter for b
alpha = 0.1 # reporting rate 10%

In [109]:
def superspreading_T_Loc(T,num_fips,initials,weights_n,pop,paras,WN):
    Z, Zb, D, Db = paras
    ## initialise 
    l0, i0 = initials
    
    NewInf = np.zeros((num_fips,T*2))
    TotInf = np.zeros((num_fips,T*2))
    
    NewInf[l0,0] = i0
    TotInf[:,0] = NewInf[:,0]
    ### for each time step and each location
    for ti in range(T):
        for l in range(num_fips):
            infectors = int(NewInf[l,ti])
            ### the number of possible new infections the size = 100 can infect individually from the NB 
            z_all = np.random.choice(len(weights_n), size = infectors, p=weights_n) ## this is a vector
            ### however, the populations have immunited people, the number will decrease i did not agree with this part 
            ## as the infection happens inside the location of the infectious people, for immunity is the pop of the 
            z_immunity = np.round(z_all*(1-TotInf[l,ti]/pop[l]))
        #     print(z_all)
        #     print(z_immunity)
        #     print('*^_^*')
            NF_l = np.zeros((2,len(z_immunity)),dtype=np.int64)
            ## for the time distribution
            latency_p = SSA.gamma.rvs(a = Z,scale=Zb,size = len(z_immunity))
            infectious_p = SSA.gamma.rvs(a = D,scale=Db,size = len(z_immunity))
            v = np.random.random_sample(len(z_immunity))
            delay_days = latency_p+v*infectious_p

            NF_l[0,:] = np.ceil(delay_days+ti) ## make it idx int
            ## for the location distribution
            loc_idx = np.random.choice(np.arange(num_fips), size = len(z_immunity), p=WN[:,l])
            NF_l[1,:] = loc_idx

            ## infections merge into the matrix
            NewInf_l = np.zeros_like(NewInf)
            for i in range(len(z_immunity)):
                t_i, loc_i = NF_l[:,i]
                NewInf_l[loc_i,t_i] = NewInf_l[loc_i,t_i]+1
            NewInf = NewInf + NewInf_l
            TotInf = np.cumsum(NewInf,axis=1)
    return NewInf, TotInf

In [110]:
NewInf, TotInf = superspreading_T_Loc(10,3,(2,1),weights_n,pop,(3,1,5,1),WN)